<a href="https://colab.research.google.com/github/hyunjin-C/hyunjin-C/blob/main/RNN_HandsOn_LM_stu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#필요한 라이브러리들 로딩하기
import tensorflow as tf

import numpy as np
import os
import time

#데이터 다운받기 (세익스피어 소설 데이터)
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# 읽은 다음 파이썬 2와 호환되도록 디코딩합니다.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#샘플 출력해보기
print(text[:20], '\n')
# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))



1122304/1115394 [==============================] - 0s 0us/step
First Citizen:
Befor 

텍스트의 길이: 1115394자


In [ ]:
# 파일의 고유 문자수를 출력 및 고유 문자로 단어장 만들기
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

# 위에서 만든 단어장을 이용해서 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

#문자 -> 인덱스 매칭 샘플 출력해보기
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


고유 문자수 65개
{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}
'First Citizen' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text) / seq_length

# 훈련 샘플/타깃 만들고 샘플 출력 해보기
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [ ]:
#seq_length+1길이만큼을 하나의 배치 단위로 데이터 만들기
#Input = sequence[:-1]
#Target = sequence[1:]
#Hello -> Input: Hell, Target: ello
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

#첫번째 다섯개 배치에 대해서 출력 해보기
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

#입력데이터랑 타겟데이터 만들기
'''
Hello 
Hell -> ello
예제
전체 문장: First Citizen
입력: First Citize
타겟: irst Citizen
'''
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


for input_example, target_example in  dataset.take(1):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
입력 데이터:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
타깃 데이터:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
   0단계
  입력: 18 ('F')
  예상 출력: 47 ('i')
   1단계
  입력: 47 ('i')
  예상 출력: 56 ('r')
   2단계
  입력: 56 ('r')
  예상 출력: 57 ('s')
   3단계
  입력: 57 ('s')
  예상 출력: 58 ('t')
   4단계
  입력: 58 ('t')
  예상 출력: 1 (' ')


In [ ]:
# 학습과 관련된 Hyper parameter들 정의 및 데이터 셔플 진행
# 배치 크기
BATCH_SIZE = 64

# 단어장의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

# 데이터셋을 섞을 버퍼 크기.
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# 모델 구조 정의 및 모델 빌드
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    ##tf.keras.layers.Embedding(input_dim, output_dim,...)
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size) #모델의 출력값은 단어장의 인덱스를 One-Hot형태로 출력하도록 학습할것이 때문에 출력 Dimension은 단어장의 크기로 설정.
  ])
  return model

model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
#데이터 샘플의 Dimension출력해보기
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

#모델 구조 출력
model.summary()

(64, 100, 65) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#랜덤하게 예측 했을 때의 결과값을 한 번 출력해보기
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

array([38, 17,  2, 61, 54, 27, 26, 20, 14, 12, 16, 37, 41, 51, 31, 22, 24,
       16,  9, 42, 24, 30, 62, 19, 27, 33,  0,  3, 28,  7, 45, 46, 54, 50,
       21, 27, 37, 58, 11, 53, 19, 60, 19, 43, 17, 17,  5, 28, 35,  5, 11,
       61, 32, 62, 39, 38, 19, 33, 58, 62, 28, 42, 32, 13, 17, 27, 20,  1,
       33,  6, 11, 16, 13, 27, 18, 22, 19, 57, 49,  9, 47, 25, 17, 35,  4,
        1, 14, 45, 22, 30,  2,  8, 13, 25, 16, 21, 62, 18, 21, 16])

In [ ]:
print("입력: \n", "".join(idx2char[input_example_batch[0]]))
print()
print("예측된 다음 문자: \n", "".join(idx2char[sampled_indices ]))

입력: 
  set of beads,
My gorgeous palace for a hermitage,
My gay apparel for an almsman's gown,
My figured 

예측된 다음 문자: 
 ZE!wpONHB?DYcmSJLD3dLRxGOU
$P-ghplIOYt;oGvGeEE'PW';wTxaZGUtxPdTAEOH U,;DAOFJGsk3iMEW& BgJR!.AMDIxFID


In [ ]:
#loss정의 및 모델 컴파일
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())


model.compile(optimizer='adam', loss=loss)

예측 배열 크기(shape):  (64, 100, 65)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           4.1739306


In [ ]:
# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
#모델 학습 진행
EPOCHS=10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 12s 53ms/step - loss: 2.6639
Epoch 2/10
172/172 [==============================] - 10s 53ms/step - loss: 1.9454
Epoch 3/10
172/172 [==============================] - 10s 53ms/step - loss: 1.6791
Epoch 4/10
172/172 [==============================] - 10s 54ms/step - loss: 1.5328
Epoch 5/10
172/172 [==============================] - 10s 54ms/step - loss: 1.4457
Epoch 6/10
172/172 [==============================] - 10s 54ms/step - loss: 1.3880
Epoch 7/10
172/172 [==============================] - 10s 54ms/step - loss: 1.3425
Epoch 8/10
172/172 [==============================] - 10s 54ms/step - loss: 1.3044
Epoch 9/10
172/172 [==============================] - 10s 54ms/step - loss: 1.2694
Epoch 10/10
172/172 [==============================] - 10s 54ms/step - loss: 1.2362


In [ ]:
#마지막 체크포인트 확인
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_10'

In [ ]:
#문장 생성을 위해서 모델의 입력 베치 사이즈를 수정하고 학습된 모델의 Weight을 읽어오기
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#언어 모델을 이용해서 입력된 단어들을 이용하여 나머지 문장들을 생성하는 함수 정의
def generate_text(model, start_string):
  # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

  # 생성할 문자의 수
  num_generate = 1000

  # 시작 문자열을 숫자로 변환(벡터화)
  input_eval = [char2idx[word] for word in start_string] ##코드 작성 필요 hint: char2idx[]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 빈 문자열
  text_generated = []

  # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
  # 온도가 높으면 더 의외의 텍스트가 됩니다.
  # 최적의 세팅을 찾기 위한 실험
  temperature = 1.0

  # 여기에서 배치 크기 == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 배치 차원 제거
      predictions = tf.squeeze(predictions, 0)

      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[0,0].numpy() ##코드 작성 필요 -> predictions 분포에서 랜덤하게 샘플링하는 코드 작성 Hint: tf.random.categorical(example_batch_predictions[0], num_samples=1)
      
      # 예측된 단어를 다음 입력으로 모델에 전달
      # 이전 은닉 상태와 함께
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
# 문장 생성 해보기
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: Kate, for't:
I thought, blows, what mine only, in arms.

KING EDWARD IV:
You are donng, sir, not op done your accusONRK:
Thou hast our senseless my last,
I do lighterous after Buckingham,
As ere thou, therefore in all this mercy, brother Marbius,
Who hastesh sin a poxt privat by
That drink no brealth. You sie is came from grief man's for.

KATHARINA:
What, his rest th Ky heart's countenance,
Thou heard'st, I pray you to the Taril: I wonder raise you: may, thou shalt not.

Nurse:
Most what they set me; this prince thou haste.

NORTHUMBERLAND:
Thy die of sweet lo, she's bagning but our company
And sayd is satch'd: if you were eyed uland,
Now come to make me breathed at my cold confised more were yet, an old heart,
Who lack as more before for I know, O, he
would his friend will kill revenge my his arm to end with solemn rawed of thine.

MISTERSER:
The messings stalls
The lady-sidow now of this old creataron:
What it will be up the unkern mourth both of this,
Which I wound her rark
